In [1]:
import requests
import re
import pandas
import time
import datetime
from bs4 import BeautifulSoup
from math import isnan
import dateutil.parser as dparser
from dateutil.parser import ParserError

In [2]:
 #Get the original page
def get_boe_page(url):
    response = requests.get(url)
    return BeautifulSoup(response.content)

In [3]:
# Scrape the state data we need to validate the form request
def get_status(base_url, fname, mname, lname, target_page):
    viewstate = target_page.find('input' , id ='__VIEWSTATE')['value']
    eventvalidation=target_page.find('input' , id ='__EVENTVALIDATION')['value']
    viewstategenerator=target_page.find('input' , id ='__VIEWSTATEGENERATOR')['value']
    params={'__EVENTTARGET':'','__EVENTARGUMENT':'','__VIEWSTATE':viewstate,
            '__VIEWSTATEGENERATOR':viewstategenerator,
            '__EVENTVALIDATION':eventvalidation,
            'cmbelectionlist':'20201103G',
            'frmLname':lname,
            'frmfname':fname,
            'btnsubmit':'Submit'}

    # Use the validation data when making the request for all current applications
    r = requests.post(base_url, data=params)
    soup = BeautifulSoup(r.text, "html.parser")
    
    results = next(iter(soup.select('#numresults p')), None)
    regex = r'Total Records found:\s?(\d+)'
    
    status = {'ballot_requested': False}
    
    if results is None:
        return status
    
    match = re.search(regex,results.text)
    num_results = int(match.group(1))
    
    if num_results == 0:
        return status
    
    rows = soup.select('tr')

    for i in range(0,num_results):
        skip = False

        for cell in rows[2+i*2].select('.oddrow~ td , .oddrow+ td , td:nth-child(1)'):
            ctype = cell['headers'][0]
            text = cell.text.strip(' \n\t\r')

            if ctype == 'Full_Name':
                if num_results == 1:
                    continue
                    
                if isinstance(mname, float) or text.find(mname) < 0:
#                     print('no matching middle name')
                    skip = True
                    break
#                 else:
#                     print('matched middle name')
            elif ctype == 'App_Date':
                status['ballot_requested'] = True
                try:
                    status['applied_date'] = dparser.parse(text)
                except ParserError:
                    status['applied_date'] = None                    
            elif ctype == 'App_Type':
                status['type'] = text.strip()
            elif ctype == 'Ballot_Processed':
                try:
                    status['send_date'] = dparser.parse(text, fuzzy=True)
                except ParserError:
                    status['received_date'] = None
            elif ctype == 'Ballot_Received':
                try:
                    status['received_date'] = dparser.parse(text, fuzzy=True)
                except ParserError:
                    status['received_date'] = None
            else:
                print(f'{ctype}: {cell}')

        if skip:
            continue

        for cell in rows[3+i*2].select('.oddrow~ td , .oddrow+ td , td:nth-child(1)'):
            ctype = cell['headers'][0]
            text = cell.text.strip(' \n\t\r')

            if ctype == 'Description':
                 status['ballot_verified'] = \
                    cell.text.find('Date Approved for Counting:') > -1
                    
    return status

In [4]:
def progressBar(iterable, prefix = '', suffix = '', decimals = 1, length = 100, numitems = 0, fill = '█', printEnd = "\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    
    total = numitems
    
    if numitems == 0:
        total = len(iterable)
        
    # Progress Bar Printing Function
    def printProgressBar (iteration):
        percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
        filledLength = int(length * iteration // total)
        bar = fill * filledLength + '-' * (length - filledLength)
        print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    # Initial Call
    printProgressBar(0)
    # Update Progress Bar
    for i, item in enumerate(iterable):
        yield i,item
        printProgressBar(i + 1)
    # Print New Line on Complete
    print()
    

In [11]:
url = 'https://lookup.boe.ohio.gov/vtrapp/athens/avlookup.aspx'
response_soup = get_boe_page(url)
get_status(base_url=url, fname='Benjamin', mname='Joseph', lname='Lachman', target_page=response_soup)

{'ballot_requested': True,
 'applied_date': datetime.datetime(2020, 8, 18, 0, 0),
 'type': 'MAIL - ABSENTEE',
 'send_date': datetime.datetime(2020, 10, 6, 0, 0),
 'received_date': datetime.datetime(2020, 10, 21, 0, 0),
 'ballot_verified': True}

In [8]:
def add_statuses(csv_filename, boe_url, response):
    statuses = list()
    df = pandas.read_csv(csv_filename)

#     num = 50
    num = len(df.index) #can cap number here
    num_skipped = 0
    
    for index, row in progressBar(df[:num].iterrows(), prefix = 'Progress:', suffix = 'Complete', numitems=num):
        row = row[1]
        
#         print(f'{index}: ' + str(row['DIDNOTSEND']) + ' / ' + str(row['RETURNED']))
        if row['DIDNOTSEND'] == 1 or row['RETURNED'] == 1:
#             print('skipping')
            num_skipped += 1
        else:
            stat = get_status(base_url=boe_url, fname=row['FIRSTN'], mname=row['MIDDLEN'], lname=row['LASTN'], target_page=response)
            statuses.append(stat)
    
    print(f'skipped: {num_skipped}')
    status_df = pandas.DataFrame.from_dict(statuses)
    
    out_df = pandas.concat([df[:num], status_df], axis=1)
    
    return out_df

In [9]:
counties = ['athens', 'hocking', 'jackson', 'meigs', 'morgan', 'vinton', 'washington']
# counties = ['morgan']
out = dict()

for county in counties:
    url = f'https://lookup.boe.ohio.gov/vtrapp/{county}/avlookup.aspx'
    csv = f'csv/{county}-thinned-final-header.csv'
    csv_out = f'csv/{county}-absentee_results-{datetime.date.today()}.csv'
    
    response_soup = get_boe_page(url)
    
    print(f'Counting results for: {county}')
    
    df = add_statuses(csv, url, response_soup)

    df.to_csv(csv_out)
    out[county] = df

Counting results for: athens
Progress: |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% Complete
skipped: 209
Counting results for: hocking
Progress: |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% Complete
skipped: 55
Counting results for: jackson
Progress: |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% Complete
skipped: 61
Counting results for: meigs
Progress: |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% Complete
skipped: 77
Counting results for: morgan
Progress: |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% Complete
skipped: 2
Counting results for: vinton
Progress: |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 

In [ ]:
def update_county(county):
    url = f'https://lookup.boe.ohio.gov/vtrapp/{county}/avlookup.aspx'
    csv = f'csv/{county}-thinned-final-header.csv'
    csv_out = f'csv/{county}-absentee_results-{datetime.date.today()}.csv'

    response_soup = get_boe_page(url)

    print(f'Counting result for: {county}')

    df = add_statuses(csv, url, response_soup)

    df.to_csv(csv_out)
    out[county] = df

In [11]:
count = 0

for key in out:
    county_count = len(out[key].index)
    print(f'{key}: ({county_count})')
    print(out[key].ballot_requested.value_counts())
    print(out[key].ballot_verified.value_counts())
    print('\n')
    count += county_count
    
print(f'total mailer count: {count}')

athens: (2100)
True     995
False    896
Name: ballot_requested, dtype: int64
True     960
False     35
Name: ballot_verified, dtype: int64


hocking: (1860)
True     1367
False     438
Name: ballot_requested, dtype: int64
True     1352
False      15
Name: ballot_verified, dtype: int64


jackson: (1860)
True     1365
False     434
Name: ballot_requested, dtype: int64
True     1326
False      39
Name: ballot_verified, dtype: int64


meigs: (1279)
True     796
False    406
Name: ballot_requested, dtype: int64
True     786
False     10
Name: ballot_verified, dtype: int64


morgan: (698)
True     512
False    184
Name: ballot_requested, dtype: int64
True    512
Name: ballot_verified, dtype: int64


vinton: (961)
True     573
False    362
Name: ballot_requested, dtype: int64
True     554
False     19
Name: ballot_verified, dtype: int64


washington: (1245)
True     620
False    509
Name: ballot_requested, dtype: int64
True    620
Name: ballot_verified, dtype: int64


total mailer count: 100

In [48]:
ocopy = dict()

for key in out:
    copy = out[key].copy()
    ocopy[key] = copy

In [49]:
for key in ocopy:
    county = ocopy[key]
    df = county[county.RETURNED != 1]
    df = df[df.DIDNOTSEND != 1]
    ocopy[key] = df
    print(len(df.index))

1744
1805
1799
1195
696
935
1129


In [50]:
count = 0

for key in ocopy:
    county_count = len(ocopy[key].index)
    print(f'{key}: ({county_count})')
    print(ocopy[key].ballot_requested.value_counts())
    print(ocopy[key].ballot_verified.value_counts())
    print('\n')
    count += county_count
    
print(f'total mailer count: {count}')

athens: (1744)
True     797
False    646
Name: ballot_requested, dtype: int64
True     770
False     27
Name: ballot_verified, dtype: int64


hocking: (1805)
True     1328
False     423
Name: ballot_requested, dtype: int64
True     1315
False      13
Name: ballot_verified, dtype: int64


jackson: (1799)
True     1322
False     417
Name: ballot_requested, dtype: int64
True     1284
False      38
Name: ballot_verified, dtype: int64


meigs: (1195)
True     740
False    376
Name: ballot_requested, dtype: int64
True     732
False      8
Name: ballot_verified, dtype: int64


morgan: (696)
True     510
False    184
Name: ballot_requested, dtype: int64
True    510
Name: ballot_verified, dtype: int64


vinton: (935)
True     552
False    357
Name: ballot_requested, dtype: int64
True     535
False     17
Name: ballot_verified, dtype: int64


washington: (1129)
True     556
False    467
Name: ballot_requested, dtype: int64
True    556
Name: ballot_verified, dtype: int64


total mailer count: 930

In [42]:
update_county('meigs')

Counting result for: meigs
Progress: |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% Complete
skipped: 84


In [47]:
update_county('athens')

Counting result for: athens
Progress: |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% Complete
skipped: 356
